In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as py


from utils.load import load_radar_dataset, load_raingauge_dataset, load_cml_dataset, get_gauge_coordinate_mappings
from models.idw import run_IDW_benchmark

In [ ]:
radar_df = load_radar_dataset(folder_name='test')
raingauge_df = load_raingauge_dataset('rainfall_data.csv', N=0)
cml_df = load_cml_dataset('CML_data_processed_2025.nc')
print(radar_df.shape)
print(raingauge_df.shape)
print(cml_df.shape)

# IDW Interpolation with rain gauge

In [ ]:
raingauge_df_5mins = raingauge_df.mul(12)
raingauge_df_15mins = raingauge_df.resample('15min').sum().mul(4) #resamples to 15 mins and converts to mm/hour rainfall rate
station_dict = get_gauge_coordinate_mappings()
idw_RMSE = run_IDW_benchmark(raingauge_df_15mins, station_dict, power=2, training_split=0.7)